In [1]:
import pandas as pd
import psycopg2
from psycopg2 import Error
import requests as re
import os
weather_key = os.environ["WETH_API"]
from collections import Counter

In [2]:
connection = psycopg2.connect(user="lhl_student",
                                  password="lhl_student",
                                  host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                                  port="5432",
                                  database="mid_term_project")
cursor=connection.cursor()

In [3]:
bigdf = pd.read_csv("C:/Users/zain_/LHL/Midterm/mid-term-project-I/fulldf.csv")

In [9]:
city_names = bigdf['origin_city_name']
flight_date = bigdf['fl_date']

In [5]:
weather_key = os.environ["WETH_API"]

In [15]:
jsonlist = []
i=0
try:
    for ele in city_names:
        url = 'http://api.weatherapi.com/v1/current.json'
        params = {'key': weather_key, 'q':ele, 'dt':flight_date[i]}
        response = re.get(url, params = params)
        i+=1
        b = response.json()
        jsonlist.append(b)
    conditions = []
    for ele in jsonlist:
        conditions.append(ele['current']['condition']['text'])
    bigdf['weather_status'] = conditions
except:
    pass

In [20]:
conditions = []
for ele in jsonlist:
    conditions.append(ele['current']['condition']['text'])

In [53]:
Counter(conditions)

Counter({'Sunny': 25148,
         'Partly cloudy': 34447,
         'Overcast': 3180,
         'Clear': 5865,
         'Mist': 753,
         'Light rain': 158,
         'Fog': 24,
         'Patchy rain possible': 11,
         'Moderate or heavy rain with thunder': 64,
         'Heavy rain': 10,
         'Patchy light rain with thunder': 92,
         'Light rain shower': 2,
         'Moderate rain': 1,
         'Thundery outbreaks possible': 8})

In [78]:
#any types with low counts merge with similar category
#make into a function, replace dataframe column with new values
adjusted_conditions = []
conditions = [x.lower() for x in conditions]
for ele in conditions:
    if 'fog' in ele:
        adjusted_conditions.append('mist')
    elif 'rain' in ele:
        adjusted_conditions.append('rain')
    elif 'thunder' in ele:
        adjusted_conditions.append('thunder')
    elif 'snow' in ele:
        adjusted_conditions.append('rain')
    elif 'sunny' in ele:
        adjusted_conditions.append('clear')
    else:
        adjusted_conditions.append(ele)

In [80]:
#data for one hot encoding
set(adjusted_conditions)

{'clear', 'mist', 'overcast', 'partly cloudy', 'rain', 'thunder'}

In [60]:
set(conditions)

{'Clear',
 'Fog',
 'Heavy rain',
 'Light rain',
 'Light rain shower',
 'Mist',
 'Moderate or heavy rain with thunder',
 'Moderate rain',
 'Overcast',
 'Partly cloudy',
 'Patchy light rain with thunder',
 'Patchy rain possible',
 'Sunny',
 'Thundery outbreaks possible'}

In [39]:
Counter(conditions)

Counter({'Sunny': 25148,
         'Partly cloudy': 34447,
         'Overcast': 3180,
         'Clear': 5865,
         'Mist': 753,
         'Light rain': 158,
         'Fog': 24,
         'Patchy rain possible': 11,
         'Moderate or heavy rain with thunder': 64,
         'Heavy rain': 10,
         'Patchy light rain with thunder': 92,
         'Light rain shower': 2,
         'Moderate rain': 1,
         'Thundery outbreaks possible': 8})

In [30]:
weather_df = bigdf[0:69763]

In [31]:
weather_df['weather_condition'] = conditions

C:\Users\zain_\AppData\Local\Temp\ipykernel_8636\2237297297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['weather_condition'] = conditions


In [32]:
weather_df['weather_condition']

0                Sunny
1                Sunny
2        Partly cloudy
3             Overcast
4        Partly cloudy
             ...      
69758            Sunny
69759    Partly cloudy
69760    Partly cloudy
69761            Sunny
69762            Sunny
Name: weather_condition, Length: 69763, dtype: object

In [81]:
main_df = weather_df

In [82]:
main_df.to_csv('main_df.csv',index=False)